In [1]:
# in this simplified version of data preprocessing, we only put images into /all_images

### Data processing

In [1]:
import os
import shutil
from PIL import Image
import random
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [29]:
root_dir = os.getcwd() + '/data_processing/'
with_mask_dir = root_dir + 'with_mask/'
without_mask_dir = root_dir + 'without_mask/'
irre_dir = root_dir + 'irrelevant/'
with_mask_resized_dir = root_dir + 'with_mask_resized/'
without_mask_resized_dir = root_dir + 'without_mask_resized/'
irre_resized_dir = root_dir + 'irrelevant_resized/'
data_dir = root_dir + 'data/'

print("Root dir: ", root_dir, "\nWith mask dir: ", with_mask_dir, " \nWithout mask dir:", without_mask_dir)
print("Data dir: ", data_dir)

Root dir:  /Users/yuchen/Desktop/CS5242/final_project/data_processing/ 
With mask dir:  /Users/yuchen/Desktop/CS5242/final_project/data_processing/with_mask/  
Without mask dir: /Users/yuchen/Desktop/CS5242/final_project/data_processing/without_mask/
Data dir:  /Users/yuchen/Desktop/CS5242/final_project/data_processing/data/


In [30]:
all_image_dir = root_dir + '/all_images/'

In [31]:
def resize(ori_dir, target_dir):
    count = 0
    for root, dirs, files in os.walk(ori_dir):
        for i, filename in enumerate(files):
#             print("processing: ", filename)
            try:
                image = Image.open(ori_dir + filename)
                n = image.resize((28, 28))
                n.save(target_dir + filename )
            except:
                print(filename)
                print("Resize failed, ready to remove")
                continue
                # os.remove(filename)
            count += 1
    print("Resize finished: ", count)

In [32]:
resize(irre_dir, irre_resized_dir)

162.jpg
Resize failed, ready to remove
176.jpg
Resize failed, ready to remove
88.jpg
Resize failed, ready to remove
2302.jpg
Resize failed, ready to remove
1151.jpg
Resize failed, ready to remove
1233.jpg
Resize failed, ready to remove
995.jpg
Resize failed, ready to remove
2288.jpg
Resize failed, ready to remove
1839.jpg
Resize failed, ready to remove
1178.jpg
Resize failed, ready to remove
2317.jpg
Resize failed, ready to remove
1352.jpg
Resize failed, ready to remove
62.jpg
Resize failed, ready to remove
1344.jpg
Resize failed, ready to remove
613.jpg
Resize failed, ready to remove
2329.jpg
Resize failed, ready to remove
411.jpg
Resize failed, ready to remove
1026.jpg
Resize failed, ready to remove
1973.jpg
Resize failed, ready to remove
1797.jpg
Resize failed, ready to remove
203.jpg
Resize failed, ready to remove
2063.jpg
Resize failed, ready to remove
772.jpg
Resize failed, ready to remove
1609.jpg
Resize failed, ready to remove
1392.jpg
Resize failed, ready to remove
1427.jpg
Re

In [33]:
len(os.listdir(irre_resized_dir))

2382

In [34]:
len(os.listdir(irre_dir))

2383

In [36]:
resize(with_mask_dir, with_mask_resized_dir)

.DS_Store
Resize failed, ready to remove
Resize finished:  2176


In [32]:
def augmentation(target_dir):
    count = 0
    for root, dirs, files in os.walk(target_dir):
        for filename in files:
            # print("processing: ", filename)
            try:
                image = Image.open(target_dir + filename)
                deg = random.randint(10, 15)
                n = image.rotate(deg)
                n.save(target_dir + "aug_" + str(deg) + "_" + filename)
                deg = random.randint(345, 350)
                n = image.rotate(deg)
                n.save(target_dir + "aug_" + str(deg) + "_" + filename)
                count += 1
            except:
                print("Skipping: ", filename)
    print("Resize finished: ", count, " Target dir: ", target_dir)

In [33]:
# augmentation(with_mask_resized_dir)

Resize finished:  2129  Target dir:  /Users/yuchen/Desktop/CS5242/final_project/data_processing/with_mask_resized/


In [34]:
# augmentation(without_mask_resized_dir)

Resize finished:  4828  Target dir:  /Users/yuchen/Desktop/CS5242/final_project/data_processing/without_mask_resized/


In [221]:
def generate_labels():
    # Calculate the files sum
    count_with_mask_files = len(os.listdir(with_mask_resized_dir))
    count_without_mask_files = len(os.listdir(without_mask_resized_dir))
    total_files = count_with_mask_files + count_without_mask_files
    print("Face with mask files: ", count_with_mask_files, " non-face files:", count_without_mask_files, " in total: ", total_files)

    label_with_mask = torch.ones(count_with_mask_files)
    label_without_mask = torch.zeros(count_without_mask_files)
    labels = torch.cat((label_without_mask, label_with_mask), 0).long()
    torch.save(labels, root_dir + 'all_labels.pt')
    print("Label writting success")
    print(len(labels))
    return count_with_mask_files, count_without_mask_files

def rename(target_dir, name_offset):
    for root, dirs, files in os.walk(target_dir):
        name = name_offset
        for filename in files:
            shutil.copyfile(target_dir + filename, data_dir + str(name) + '.jpg')
            name += 1
        print("Renaming done")

In [222]:
# c_with, c_without = generate_labels()

Face with mask files:  2182  non-face files: 4649  in total:  6831
Label writting success
6831


In [224]:
# rename(without_mask_resized_dir, 0)
# rename(with_mask_resized_dir, c_without)

Renaming done
Renaming done


In [37]:
def rename_with_label(cur_dir, target_dir, suffix):
    for root, dirs, files in os.walk(cur_dir):
        for filename in files:
            new_name = filename[:-4] + suffix + ".jpg"
            shutil.copyfile(cur_dir + filename, target_dir + new_name)
        print("Renaming done")

In [40]:
rename_with_label(with_mask_resized_dir, all_image_dir, "_1")
rename_with_label(without_mask_resized_dir, all_image_dir, "_0")
rename_with_label(irre_resized_dir, all_image_dir, "_2")

Renaming done


### Data split

In [42]:
import os
import shutil
from PIL import Image
import random
import torch

In [43]:
os.getcwd()

'/Users/yuchen/Desktop/CS5242/final_project'

In [44]:
all_images = os.listdir(all_image_dir)

In [45]:
len(all_images)

9207

In [46]:
## Set up folders
def ensure_folder(folder_name):
    try:
        os.makedirs(folder_name)
    except FileExistsError:
        pass

root_dir = os.getcwd() + '/data_processing/'
train_set = root_dir + 'train_set_with_label/'
val_set = root_dir + 'val_set_with_label/'
test_set = root_dir + 'test_set_with_label/'

# Create folders
ensure_folder(train_set)
ensure_folder(val_set)
ensure_folder(test_set)

In [47]:
data_size = len(all_images)
print("Find data count: %s, Ready to split "% data_size)

Find data count: 9207, Ready to split 


In [48]:
# Start to split data
val_size = int(data_size * 0.15)
test_size = int(data_size * 0.15)
train_size = data_size - val_size - test_size
print("Val size:",val_size, ", Test size:", test_size, ", train size:", (data_size - val_size - test_size))

Val size: 1381 , Test size: 1381 , train size: 6445


In [50]:
random.shuffle(all_images)

In [51]:
all_images

['60_2.jpg',
 '2075_0.jpg',
 '1338_2.jpg',
 '603_2.jpg',
 '4801_0.jpg',
 '502_0.jpg',
 '251_0.jpg',
 '3799_0.jpg',
 '543_2.jpg',
 '1466_1.jpg',
 '954_0.jpg',
 '4295_0.jpg',
 '2975_0.jpg',
 '905_1.jpg',
 '751_1.jpg',
 '1672_2.jpg',
 '2055_0.jpg',
 '4684_0.jpg',
 '204_0.jpg',
 '1297_2.jpg',
 '176_0.jpg',
 '4560_0.jpg',
 '1506_0.jpg',
 '680_0.jpg',
 '437_0.jpg',
 '1473_1.jpg',
 '4252_1.jpg',
 '1930_0.jpg',
 '1094_2.jpg',
 '3920_0.jpg',
 '3642_0.jpg',
 '3721_0.jpg',
 '1116_0.jpg',
 '2282_0.jpg',
 '958_0.jpg',
 '1440_0.jpg',
 '1452_2.jpg',
 '2_0.jpg',
 '1308_1.jpg',
 '3401_0.jpg',
 '3751_0.jpg',
 '510_1.jpg',
 '2881_0.jpg',
 '684_2.jpg',
 '1500_0.jpg',
 '1028_2.jpg',
 '693_2.jpg',
 '1814_2.jpg',
 '1933_2.jpg',
 '1080_2.jpg',
 '408_2.jpg',
 '1022_2.jpg',
 '626_1.jpg',
 '991_1.jpg',
 '270_1.jpg',
 '4735_0.jpg',
 '1792_1.jpg',
 '1630_2.jpg',
 '1085_0.jpg',
 '1730_0.jpg',
 '2134_2.jpg',
 '1486_2.jpg',
 '3918_0.jpg',
 '197_2.jpg',
 '1747_2.jpg',
 '1745_1.jpg',
 '2230_2.jpg',
 '3441_0.jpg',
 '306

In [54]:
for i, image in enumerate(all_images):
    if i < val_size:
        shutil.copyfile(all_image_dir + image, val_set + image)
    elif i < val_size + test_size:
        shutil.copyfile(all_image_dir + image, test_set + image)
    else:
        shutil.copyfile(all_image_dir + image, train_set + image)

In [55]:
len(os.listdir(val_set))

1381

In [56]:
len(os.listdir(test_set))

1381

In [57]:
len(os.listdir(train_set))

6445